In [ ]:
import sys
sys.path.append(r'C:/Program Files (zk)/PythonFiles/AClassification/SoundDL-CoughVID/')
# sys.path.append(r'D:/PythonTorchVITS/CoughDL/SoundDL-CoughVID/')
import os
import numpy as np
import librosa
import pandas as pd
import IPython.display as ipd
import matplotlib.pyplot as plt
from readers.audio import AudioSegment
# root_path = "F:/DATAS/COUGHVID-public_dataset_v3/coughvid_20211012/"

In [ ]:
root_path = "F:/DATAS/COUGHVID-public_dataset_v3/coughvid_20211012/"
metafile = "F:/DATAS/COUGHVID-public_dataset_v3/coughvid_20211012/metadata_compiled.csv"

# 去除静音片段然后计算时长然后保存标签

In [ ]:
metadf = pd.read_csv(metafile, delimiter=',', header=0, index_col=0)
# print(metadf.head(5))

fname_list = []
dura_list = []
anomaly_list = []
status_id = []
status_SSL_id = []
status2label = {"healthy": 0, "symptomatic": 1, "COVID-19": 2}
for i, row in enumerate(metadf.itertuples()):
    wi = getattr(row, "uuid")
    fname = root_path + wi
    for ext in ["webm", "wav", "ogg"]:
        if os.path.exists(fname+'.'+ext):
            fname = fname + '.' + ext
    fname_list.append(fname)
    try:
        audioseg = AudioSegment.from_file(file=fname)
    except Exception as e:
        print(e)
        print("Read Error:", fname, "skip it and continue.")
    audioseg.vad()
    dura_list.append(audioseg.duration)
    ano = -1
    sta_str = getattr(row, "status")
    if isinstance(sta_str, str):
        sid = status2label[sta_str]
        status_id.append(sid)
        if sid == 0:
            ano = 0
        else:
            ano = 1
    else:
        status_id.append(sta_str)
    stassl_str = getattr(row, "status_SSL")
    if isinstance(stassl_str, str):
        sid = status2label[stassl_str]
        status_SSL_id.append(sid)
        if ano == -1:
            if sid == 0:
                ano = 0
            else:
                ano = 1
    else:
        status_SSL_id.append(stassl_str)
    if ano == -1:
        ano = float('nan')
    anomaly_list.append(ano)
    if i%300==0:
        print(i, ':', fname)
    # if i==19:
    #     break


In [ ]:
wavedura = metadf.iloc[:, [0,2]]  # ,9,10]]
wavedura["uuid"] = fname_list
wavedura["duration"] = dura_list
wavedura["status_id"] = status_id
wavedura["status_SSL_id"] = status_SSL_id
wavedura["anomaly"] = anomaly_list
wavedura.columns = ["filename", "cough_detected", "duration", "status", "status_SSL", "anomaly"]
print(wavedura.head(20))
print(len(wavedura))  # .head(20)

In [ ]:
wavedura.to_csv("C:/Program Files (zk)/PythonFiles/AClassification/SoundDL-CoughVID/datasets/waveinfo.csv", sep=',')

# 把具有异常标注的音频保存下来

In [ ]:
waveinfo_labeled_indices = []
waveinfo_unlabeled_indices = []
i=0
with open("./datasets/waveinfo.csv", 'r') as fin:
    fin.readline()
    line = fin.readline().strip()
    while line:
        parts = line.split(',')
        # print(parts)
        wi = parts[6]
        print(wi, end=', ')
        if len(wi) > 0:
            # waveinfo_labeled_indices.append(parts[0])
            waveinfo_labeled_indices.append(i)
        else:
            # waveinfo_unlabeled_indices.append(parts[0])
            waveinfo_unlabeled_indices.append(i)
        # if i > 40:
        #     break
        i += 1
        line = fin.readline().strip()
print()
print(waveinfo_labeled_indices)
print(waveinfo_unlabeled_indices)

In [ ]:
metadf = pd.read_csv("./datasets/waveinfo.csv", delimiter=',', header=0, index_col=0)
waveinfo_labeled = metadf.iloc[waveinfo_labeled_indices, :]
waveinfo_unlabeled = metadf.iloc[waveinfo_unlabeled_indices, :]
# waveinfo_unlabeled = waveinfo_unlabeled[waveinfo_unlabeled["anomaly"]!=0]
print(len(waveinfo_labeled))
print(len(waveinfo_unlabeled))

In [ ]:
waveinfo_labeled.to_csv("C:/Program Files (zk)/PythonFiles/AClassification/SoundDL-CoughVID/datasets/waveinfo_labeled.csv", sep=',')
waveinfo_unlabeled.to_csv("C:/Program Files (zk)/PythonFiles/AClassification/SoundDL-CoughVID/datasets/waveinfo_unlabeled.csv", sep=',')

## 统计每个标签的数目

In [ ]:
datadf1 = pd.read_csv("F:/DATAS/COUGHVID-public_dataset_v3/waveinfo.csv", delimiter=',', header=0, index_col=0)
datadf1.groupby("status")["filename"].count()

# 加上扩展名

In [ ]:

filename = []
for i, uuid in enumerate(datadf1.iloc[:, 0]):
    for ext in ["webm", "wav", "ogg"]:
        fname = root_path + uuid + '.' + ext
        if os.path.exists(fname):
            filename.append(fname)
print("end")

# datadf1["filename"] = filename
# datadf2 = datadf1.iloc[:, [5,1,2,3,4]]
# datadf2.to_csv("F:/DATAS/COUGHVID-public_dataset_v3/waveinfo.csv", sep=',')

# 合并新的列 已完成

In [ ]:
datadf1 = pd.read_csv("F:/DATAS/COUGHVID-public_dataset_v3/waveinfo.csv", delimiter=',', header=0, index_col=0)
datadf2 = pd.read_csv("F:/DATAS/COUGHVID-public_dataset_v3/coughvid_20211012/metadata_compiled.csv", delimiter=',',
                 header=0, index_col=0)
# print(datadf1.head(15))
print("row num:", len(datadf1), len(datadf2))
datadf2 = datadf2.iloc[:, [0,9]]
datadf1 = datadf1.where(datadf1["cough_detected"]>0.65).where(datadf1["duration"] < 13).where(datadf1["duration"] > 0.86).dropna(axis=0)

In [ ]:
datadf3 = datadf1.set_index("uuid").join(datadf2.set_index("uuid"), on="uuid", how="left")
print(len(datadf3))
datadf3

In [ ]:
datadf3.to_csv("F:/DATAS/COUGHVID-public_dataset_v3/waveinfo.csv", sep=',')

## 排序和where筛选

In [ ]:
datadf1 = datadf1.where(datadf1["cough_detected"]>0.65).where(datadf1["duration"] < 13).where(datadf1["duration"] > 0.86).dropna(axis=0)
datadf1 = datadf1.sort_values("uuid", ascending=False)
datadf1.head(15)

# 波形图绘图

In [ ]:
x, sr = librosa.load(root_path + "ffe8b243-9c7b-49f1-8d7d-a953f736ea4b.wav")
print("sample rate:", sr)
# print("filename:", root_path+df.iloc[0, :][0])
# ipd.Audio(x, rate=sr)

In [ ]:
plt.figure(0)
plt.plot(range(len(x)), x, color="black")
plt.show()

## 读取部分列

In [ ]:
metadata = pd.read_csv("F:/DATAS/COUGHVID-public_dataset_v3/coughvid_20211012/metadata_compiled.csv", delimiter=',',
                 header=0, index_col=0)
datadf1 = metadata.iloc[:, [0,2,9]]
datadf1 = datadf1.sort_values("status", ascending=True)
datadf1.head(15)

## 已经合并得到的信息↓，不再需要运行

In [ ]:
df = pd.read_csv("F:/DATAS/COUGHVID-public_dataset_v3/dura_stat.csv", delimiter='\t', header=None, index_col=None)
df.columns = ["filename", "nframes", "duration"]
df = df.sort_values("nframes", ascending=True)
print(df.head(15))

In [ ]:
metadata = pd.read_csv("F:/DATAS/COUGHVID-public_dataset_v3/coughvid_20211012/metadata_compiled.csv", delimiter=',',
                 header=0, index_col=0)
datadf1 = metadata.iloc[:, [0,2]]
datadf1 = datadf1.sort_values("uuid", ascending=True)
datadf1.head(15)

datadf2 = pd.read_csv("F:/DATAS/COUGHVID-public_dataset_v3/dura_stat.csv", delimiter='\t', header=None, index_col=None)
datadf2.columns = ["uuid", "nframes", "duration"]
datadf2 = datadf2.sort_values("uuid", ascending=True)
datadf2.head(15)

datadf1["nframes"] = datadf2["nframes"]
datadf1["duration"] = datadf2["duration"]


In [ ]:
datadf1 = datadf1.sort_values("cough_detected", ascending=True)
datadf1.to_csv("F:/DATAS/COUGHVID-public_dataset_v3/waveinfo.csv", sep=',')

In [ ]:
x, sr = librosa.load(root_path + "00bfe21c-ab71-4e5a-a941-4f83f5de5c82.wav")
print("sample rate:", sr)
print("filename:", root_path+df.iloc[0, :][0])
ipd.Audio(x, rate=sr)

In [ ]:
datadf1 = datadf1.sort_values("duration", ascending=False)
datadf1.where(datadf1["cough_detected"]>0.65).where(datadf1["duration"] < 13).where(datadf1["duration"] > 0.86).dropna(axis=0)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(0)
plt.scatter(datadf1["cough_detected"], datadf2["duration"])
plt.show()